In [1]:
%load_ext autoreload
%autoreload 2

# Database Controller

In [2]:
from urllib.parse import urljoin
from dotenv import load_dotenv
from bson import ObjectId
import pandas as pd
import numpy as np
import requests
import json
import os
load_dotenv()

True

In [3]:
class GSHubAPI:
    def __init__(self) -> None:
        self.domain: str = "https://premo.gshub.nl/"
    
    def learn_url(self, days_in_past: int) -> str:
        return urljoin(self.domain, f"api/dinand/learn/{days_in_past}")
    
    @property
    def play_url(self) -> str:
        return urljoin(self.domain, "api/dinand/play/")

In [4]:
learn_url = GSHubAPI().learn_url(7000)
learn_url

'https://premo.gshub.nl/api/dinand/learn/7000'

In [5]:
from modules.database_tables import Database
from modules.database_tables.Orders import Orders
from modules.database_tables.Products import Products
from modules.database_tables.TaskCards import TaskCards
from modules.database_tables.ProductOperations import Operations

In [6]:
username = os.getenv("mongodb_user")
password = os.getenv("mongodb_password")

In [7]:
db = Database(username, password)
orders = Orders(username, password)
products = Products(username, password)
taskcards = TaskCards(username, password)
operations = Operations(username, password)

## Insert Order Data into Database

In [8]:
data = requests.get(learn_url).json()
len(data)

132

In [9]:
db.db.drop_collection('orders')
db.db.drop_collection('products')
db.db.drop_collection('task_cards')
db.db.drop_collection('operations')

{'ok': 1.0,
 '$clusterTime': {'clusterTime': Timestamp(1727789302, 25),
  'signature': {'hash': b'h]\x1f_P\x9c\xd9r\x93q\xa7\x04\xb2?\xaa\x95<1]p',
   'keyId': 7376309071898476564}},
 'operationTime': Timestamp(1727789302, 25)}

In [10]:
orders_bulk = []
products_bulk = []
taskcards_bulk = []
operations_bulk = []

for order in data:
    order_dict = {k: v for k, v in order.items() if k in ['order_id', 'delivery_date', 'order_created_at']}
    if orders.find({'order_id': order_dict['order_id']}): continue
    order_id = orders.insert(order_dict)
    
    for prod in order['products']:
        products_dict = {k: v for k, v in prod.items() if k in ['material', 'color']}
        products_dict['order_id'] = order_id
        product_id = products.insert(products_dict)
        
        taskcards_arr: list = [{**taskcard, 'product_id': product_id} for taskcard in prod['task_cards']]
        taskcards_bulk.extend(taskcards_arr)
        
        operations_arr = [{**operation, 'product_id': product_id} for operation in prod['product_operations']]
        operations_bulk.extend(operations_arr)
        

# Bulk insert taskcards and operations
if len(taskcards_bulk) > 0 and len(operations_bulk) > 0:
    tk_ids: list[ObjectId] = taskcards.insert_many(taskcards_bulk)
    op_ids: list[ObjectId] = operations.insert_many(operations_bulk)